In [2]:
import pandas as pd
import os # Biblioteca para interagir com o sistema operacional
import re

In [3]:
# ---
# Passo 1: Definir o caminho para a pasta com os dados brutos
caminho_dados_brutos = '/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/doencas-datasus/'

# Lista para armazenar os DataFrames de cada doença processada
lista_dfs_doencas = []

print("Iniciando processamento dos arquivos do DATASUS...")

Iniciando processamento dos arquivos do DATASUS...


In [4]:
# Passo 2: Loop para processar cada arquivo CSV na pasta
for filename in os.listdir(caminho_dados_brutos):
    if filename.endswith('.csv'):
        caminho_completo = os.path.join(caminho_dados_brutos, filename)
        print(f"  Lendo o arquivo: {filename}")

        # Pega a terceira parte do nome do arquivo, ex: 'leptope14474145'
        parte_doenca = filename.split('_')[2]
        # Usa regex para extrair apenas as letras do início da string
        # e remove 'pe' e 'b' para padronizar
        match = re.search(r'([a-zA-Z]+)', parte_doenca)
        if match:
            nome_doenca = match.group(1).replace('pe', '').replace('dengueb', 'dengue')

        try:
            # ---
            # *** LÓGICA CONDICIONAL ADICIONADA AQUI ***
            # Se o arquivo for do 'sinannet', pulamos 5 linhas. Caso contrário, pulamos 4.
            if 'sinannet' in filename:
                linhas_a_pular = 5
            else: # para 'sinanwin' e qualquer outro
                linhas_a_pular = 4

            print(f"    -> Arquivo do tipo '{'sinannet' if 'sinannet' in filename else 'sinanwin'}'. Pulando {linhas_a_pular} linhas.")

            df_raw = pd.read_csv(
                caminho_completo,
                encoding='latin-1',
                sep=';',
                skiprows=linhas_a_pular, # Usando a variável que definimos
                skipfooter=3,
                engine='python',
                decimal=','
            )

            # ---
            # Passo 3: Limpeza da coluna de municípios (com Regex, que já é robusto)
            coluna_municipio_raw_nome = df_raw.columns[0]
            df_raw.rename(columns={coluna_municipio_raw_nome: 'municipio_raw'}, inplace=True)

            df_raw['cod_municipio_6dig'] = df_raw['municipio_raw'].str.extract(r'(\d{6})', expand=False)
            df_raw['nome_municipio'] = df_raw['municipio_raw'].str.replace(r'^\d{6}\s*', '', regex=True)

            df_raw.drop(columns=['municipio_raw'], inplace=True)
            df_raw.dropna(subset=['cod_municipio_6dig'], inplace=True)

            # ---
            # Passo 4: Transpor os dados (de formato largo para longo)
            df_long = pd.melt(
                df_raw,
                id_vars=['cod_municipio_6dig', 'nome_municipio'],
                var_name='ano',
                value_name='casos_notificados'
            )

            df_long['doenca'] = nome_doenca
            lista_dfs_doencas.append(df_long)
            print(f"    -> {filename} processado com sucesso.")

        except Exception as e:
            print(f"    -> ERRO ao processar o arquivo {filename}: {e}")

  Lendo o arquivo: sinanwin_cnv_leptope14474145_4_61_174.csv
    -> Arquivo do tipo 'sinanwin'. Pulando 4 linhas.
    -> sinanwin_cnv_leptope14474145_4_61_174.csv processado com sucesso.
  Lendo o arquivo: sinanwin_cnv_denguepe15062545_4_61_174.csv
    -> Arquivo do tipo 'sinanwin'. Pulando 4 linhas.
    -> sinanwin_cnv_denguepe15062545_4_61_174.csv processado com sucesso.
  Lendo o arquivo: sinanwin_cnv_esquistope14591445_4_61_174.csv
    -> Arquivo do tipo 'sinanwin'. Pulando 4 linhas.
    -> sinanwin_cnv_esquistope14591445_4_61_174.csv processado com sucesso.
  Lendo o arquivo: sinanwin_cnv_febretifoidepe14573145_4_61_174.csv
    -> Arquivo do tipo 'sinanwin'. Pulando 4 linhas.
    -> sinanwin_cnv_febretifoidepe14573145_4_61_174.csv processado com sucesso.
  Lendo o arquivo: sinanwin_cnv_hepape14553745_4_61_174.csv
    -> Arquivo do tipo 'sinanwin'. Pulando 4 linhas.
    -> sinanwin_cnv_hepape14553745_4_61_174.csv processado com sucesso.
  Lendo o arquivo: sinannet_cnv_leptope151332

In [5]:
# ---
# Passo 5: Concatenar todos os dataframes de doenças em um único
print("\nConcatenando todos os dados de saúde...")
df_saude_completo = pd.concat(lista_dfs_doencas, ignore_index=True)

# Limpeza final
df_saude_completo['ano'] = pd.to_numeric(df_saude_completo['ano'], errors='coerce')
df_saude_completo.dropna(subset=['cod_municipio_6dig', 'ano'], inplace=True)
df_saude_completo['cod_municipio_6dig'] = df_saude_completo['cod_municipio_6dig'].astype(int)
df_saude_completo['ano'] = df_saude_completo['ano'].astype(int)

print("\n--- Processamento de Saúde Concluído! ---")
print("Amostra da base de dados de saúde final:")
print(df_saude_completo.head())


Concatenando todos os dados de saúde...

--- Processamento de Saúde Concluído! ---
Amostra da base de dados de saúde final:
   cod_municipio_6dig nome_municipio   ano casos_notificados doenca
0              260005   ABREU E LIMA  2001                 7  lepto
1              260030      AGRESTINA  2001                 -  lepto
2              260040     AGUA PRETA  2001                 -  lepto
3              260050    AGUAS BELAS  2001                 -  lepto
4              260070        ALIANCA  2001                 1  lepto


In [6]:
print("--- Iniciando Limpeza Final e Balanceamento do Painel ---")

# ---
# ETAPA 1: Corrigir a coluna 'casos_notificados'
# ---
print("\nPasso 1: Substituindo '-' por 0 e convertendo para inteiro...")

# Substituir o caractere '-' pelo número 0
df_saude_completo['casos_notificados'] = df_saude_completo['casos_notificados'].replace('-', 0)

# Converter a coluna para um tipo numérico (float primeiro para lidar com possíveis NaNs)
df_saude_completo['casos_notificados'] = pd.to_numeric(df_saude_completo['casos_notificados'], errors='coerce')

# Preencher quaisquer outros valores nulos que possam ter surgido
df_saude_completo['casos_notificados'] = df_saude_completo['casos_notificados'].fillna(0)

# Finalmente, converter para inteiro (int)
df_saude_completo['casos_notificados'] = df_saude_completo['casos_notificados'].astype(int)

print("Coluna 'casos_notificados' limpa e convertida para inteiro.")
print(df_saude_completo.info())

--- Iniciando Limpeza Final e Balanceamento do Painel ---

Passo 1: Substituindo '-' por 0 e convertendo para inteiro...
Coluna 'casos_notificados' limpa e convertida para inteiro.
<class 'pandas.core.frame.DataFrame'>
Index: 18242 entries, 0 to 19853
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   cod_municipio_6dig  18242 non-null  int64 
 1   nome_municipio      18242 non-null  object
 2   ano                 18242 non-null  int64 
 3   casos_notificados   18242 non-null  int64 
 4   doenca              18242 non-null  object
dtypes: int64(3), object(2)
memory usage: 855.1+ KB
None


In [7]:
# ---
# ETAPA 2: Criar um painel balanceado (a sua estratégia para linhas vazias)
# ---
print("\nPasso 2: Criando um painel balanceado para incluir observações de 'zero casos'...")

# A. Criar a lista completa de todas as entidades do seu painel
anos = range(2002, 2022) # Seu período de interesse
municipios_unicos = df_saude_completo[['cod_municipio_6dig', 'nome_municipio']].drop_duplicates()
doencas_unicas = df_saude_completo['doenca'].unique()

# B. Criar o "molde" ou "scaffold" com todas as combinações possíveis
# Usando pd.MultiIndex.from_product para criar todas as combinações
idx = pd.MultiIndex.from_product([anos, municipios_unicos['cod_municipio_6dig'], doencas_unicas],
                                 names=['ano', 'cod_municipio_6dig', 'doenca'])
painel_completo = pd.DataFrame(index=idx).reset_index()

# Adicionar o nome do município de volta ao molde
# Criar um dicionário de mapeamento: codigo -> nome
mapa_nomes = municipios_unicos.set_index('cod_municipio_6dig')['nome_municipio']
painel_completo['nome_municipio'] = painel_completo['cod_municipio_6dig'].map(mapa_nomes)


# C. Fazer um 'left merge' dos seus dados de saúde no molde completo
# O molde está à esquerda, garantindo que todas as combinações de ano/município/doença sejam mantidas
df_final_balanceado = pd.merge(
    painel_completo,
    df_saude_completo,
    on=['ano', 'cod_municipio_6dig', 'doenca', 'nome_municipio'],
    how='left'
)

# D. Preencher os valores vazios (NaN) com zero
# As linhas que existiam no molde mas não nos seus dados são os casos de "zero ocorrências"
df_final_balanceado['casos_notificados'] = df_final_balanceado['casos_notificados'].fillna(0)

# Converter a coluna para inteiro novamente, pois o merge pode ter reintroduzido floats
df_final_balanceado['casos_notificados'] = df_final_balanceado['casos_notificados'].astype(int)

print("\n--- Painel Balanceado Concluído! ---")
print(f"Número de linhas original: {len(df_saude_completo)}")
print(f"Número de linhas no painel balanceado: {len(df_final_balanceado)}")

print("\nAmostra da base de dados final e balanceada:")
# Vamos ver um exemplo para um município/doença e ver se todos os anos estão presentes
print(df_final_balanceado[
    (df_final_balanceado['cod_municipio_6dig'] == 260030) & # Agrestina
    (df_final_balanceado['doenca'] == 'lepto')
].head())

df_final_balanceado.to_csv('/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/output/doencas_pe_2001_2021.csv', index=False, encoding='utf-8-sig')


Passo 2: Criando um painel balanceado para incluir observações de 'zero casos'...

--- Painel Balanceado Concluído! ---
Número de linhas original: 18242
Número de linhas no painel balanceado: 27195

Amostra da base de dados final e balanceada:
       ano  cod_municipio_6dig doenca nome_municipio  casos_notificados
7     2002              260030  lepto      AGRESTINA                  0
1302  2003              260030  lepto      AGRESTINA                  2
2597  2004              260030  lepto      AGRESTINA                  0
3892  2005              260030  lepto      AGRESTINA                  2
5187  2006              260030  lepto      AGRESTINA                  1


In [ ]:
# Carregar a base socioeconômica para obter a população anual
try:
    df_socioeconomico = pd.read_csv('/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/output/pib_dens_hab_pe_2002_2021.csv')
    # Selecionar apenas as colunas que precisamos para a junção
    df_populacao = df_socioeconomico[['cod_municipio', 'ano', 'populacao_estimada']].copy()
    print("Arquivo 'pib_dens_hab_pe_2002_2021.csv' carregado com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivo 'pib_dens_hab_pe_2002_2021.csv' não encontrado.")
    # Adicione um 'return' ou 'exit()' aqui.

# ---
# ETAPA 2: Agregar as Arboviroses
# ---
print("\nAgregando os casos de Dengue, Chikungunya e Zika em 'Arboviroses'...")

# Lista das doenças que compõem as arboviroses
lista_arboviroses = ['dengue', 'chikun', 'zika']

# Filtrar o DataFrame para conter apenas as arboviroses
df_arboviroses_raw = df_final_balanceado[df_final_balanceado['doenca'].isin(lista_arboviroses)]

# Agrupar por município e ano e somar os casos
df_arboviroses_agregado = df_arboviroses_raw.groupby(['cod_municipio_6dig', 'nome_municipio', 'ano'])['casos_notificados'].sum().reset_index()

# Adicionar uma coluna para identificar a nova categoria de doença
df_arboviroses_agregado['doenca'] = 'arboviroses'

# ---
# ETAPA 3: Isolar as outras doenças e juntar tudo
# ---
print("Isolando as outras doenças de interesse e combinando os DataFrames...")

# Lista das outras doenças que queremos manter separadas
lista_outras_doencas = ['esquisto', 'lepto', 'febretifoide'] # Certifique-se que os nomes estão corretos

# Filtrar o DataFrame original para manter apenas essas doenças
df_outras_doencas = df_final_balanceado[df_final_balanceado['doenca'].isin(lista_outras_doencas)]

# Concatenar o novo DataFrame de arboviroses com o das outras doenças
df_saude_processado = pd.concat([df_arboviroses_agregado, df_outras_doencas], ignore_index=True)

In [24]:
# ---
# ETAPA 4: Calcular a Taxa de Incidência por 100 mil Habitantes
# ---
print("Juntando com os dados de população e calculando as taxas de incidência...")

# Renomear a coluna de código do município para o merge
df_populacao['cod_municipio_6dig'] = df_populacao['cod_municipio'] // 10

# Unir os dados de saúde com os de população
df_com_pop = pd.merge(
    df_saude_processado,
    df_populacao,
    on=['cod_municipio_6dig', 'ano'],
    how='left' # Usar 'left' para manter todos os registros de saúde
)

# Calcular a taxa, tratando o caso de população ser zero ou nula
df_com_pop['taxa_100mil_hab'] = 0.0
# Evitar divisão por zero
mask_pop_valida = df_com_pop['populacao_estimada'] > 0
df_com_pop.loc[mask_pop_valida, 'taxa_100mil_hab'] = \
    (df_com_pop['casos_notificados'] / df_com_pop['populacao_estimada']) * 100000

Juntando com os dados de população e calculando as taxas de incidência...


In [ ]:
# ---
# ETAPA 5: Pivotar a Tabela para o Formato Final de Análise
# ---
print("Pivotando a tabela para ter uma coluna por doença...")

df_saude_final = df_com_pop.pivot_table(
    index=['cod_municipio', 'ano'],
    columns='doenca',
    values='taxa_100mil_hab'
).reset_index()

# Limpar os nomes das colunas após o pivot
df_saude_final.columns.name = None
# Renomear as colunas para um formato mais limpo (ex: 'taxa_arboviroses')
df_saude_final.columns = [f'taxa_{col}' if col not in ['cod_municipio', 'ano'] else col for col in df_saude_final.columns]

print("\n--- Processamento de Saúde Finalizado! ---")
print("Amostra da base de dados de saúde final, pronta para o merge:")
print(df_saude_final.head())

df_saude_final.info()

df_saude_final.to_parquet('/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/output/doencas_pe_2002_2021.parquet', index=False)